# Part 1: Scraping data from Wikipidea

### Import Libraries

In [1]:
import requests # library to handle requests
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

### Extract postal codes

In [2]:
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"  # Wiki url
col = 3  # 3 col data
page  = urllib.request.urlopen(url).read()
soup  = bs.BeautifulSoup(page,'html.parser')
table = soup.find("table",class_='wikitable')
header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
data    = [row for row in data if len(row) == col]
    
temp_df = pd.DataFrame(data,columns=header)   # get temp data

 ### Process the temporary data

In [3]:
df=temp_df            
df = df[df.Borough != 'Not assigned']  
# intergrate postal codes with more than one neighbour 
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
print(df.shape)
df.head()         # check processed data

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Save Toronto postcode data

In [4]:
df.to_csv('Toronto Postcode.csv',index=False)